In [1]:
%matplotlib inline

In [2]:
import pandas as pd
from psyrun import NpzStore

In [3]:
data = NpzStore().load('../data/memory_spaun.npz')

In [4]:
import xarray as xr

In [5]:
data.keys()

dict_keys(['trial', 'backend', 'time', 'event_times', 'memory', 'model'])

In [6]:
data['event_times'].shape

(20, 3)

In [7]:
np.unique(data['model'])

array(['spaun'], 
      dtype='<U5')

In [8]:
ds = xr.Dataset({
    'memory': (['idx', 't_idx'], data['memory']),
    'time': (['idx', 't_idx'], data['time']),
    'trial': (['idx'], data['trial']),
    'backend': (['idx'], data['backend']),
    'model': (['model'], data['model']),
}, coords={'idx': range(len(data['trial'])), 't_idx': range(data['time'].shape[1])})

In [9]:
def get_coords(x):
    coords = np.unique(x)
    try:
        coords[np.logical_not(np.isfinite(coords))] = -1
    except:
        pass
    return np.unique(coords)

keys = ['model', 'backend', 'trial']
coords = {k: get_coords(data[k]) for k in keys}
coords['t_idx'] = range(data['time'].shape[1])
shape = tuple(len(coords[k]) for k in keys) + (data['time'].shape[1],)

In [10]:
ds = xr.Dataset({
        'memory': (keys + ['t_idx'], np.empty(shape)),
        't': (keys + ['t_idx'], np.empty(shape)),
        'events': (keys + ['ev'], np.empty(shape[:-1] + (3,)))
    }, coords=coords)
ds

<xarray.Dataset>
Dimensions:  (backend: 4, ev: 3, model: 1, t_idx: 79498, trial: 5)
Coordinates:
  * t_idx    (t_idx) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * trial    (trial) int64 0 1 2 3 4
  * backend  (backend) <U9 'ocl_cpu' 'ocl_gpu' 'optimized' 'reference'
  * model    (model) <U5 'spaun'
  * ev       (ev) int64 0 1 2
Data variables:
    t        (model, backend, trial, t_idx) float64 0.0 0.0 0.0 0.0 0.0 0.0 ...
    memory   (model, backend, trial, t_idx) float64 0.0 0.0 0.0 0.0 0.0 0.0 ...
    events   (model, backend, trial, ev) float64 6.93e-310 6.93e-310 ...

In [11]:
import itertools
for x in itertools.product(*(coords[k] for k in keys)):
    sel = np.ones_like(data['memory'], dtype=bool)
    for i, k in enumerate(keys):
        sel = np.logical_and(sel, (data[k] == x[i])[:, None])
        #print(k, x[i])
        #print(np.any(data[k] == x[i]))
        #print(np.any(sel))
    loc = {k: x[i] for i, k in enumerate(keys)}
    #print(data['memory'].shape, sel.shape, sel)
    if np.any(sel):
        ds['memory'].loc[loc] = data['memory'][sel]
        ds['t'].loc[loc] = data['time'][sel]
        ds['events'].loc[loc] = data['event_times'][np.max(sel, axis=-1)]

In [12]:
da = ds.loc[{
    'model': 'spaun',
    'backend': 'reference',
}]
print('reference:', np.max(da['memory'] / 1024. / 1024.))

da = ds.loc[{
    'model': 'spaun',
    'backend': 'optimized',
}]
print('optimized:', np.max(da['memory'] / 1024. / 1024.))

reference: <xarray.DataArray 'memory' ()>
array(12248.0234375)
Coordinates:
    backend  <U9 'reference'
    model    <U5 'spaun'
optimized: <xarray.DataArray 'memory' ()>
array(19111.60546875)
Coordinates:
    backend  <U9 'optimized'
    model    <U5 'spaun'
